In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 999)

In [2]:
# Importing the part df on local and full data on cloud
#filename = '/Users/viswanathravindran/AVDatahack/ADClick/Data/train.csv'
#Traindf = pd.read_csv(filename, nrows = 80000)
Traindf = pd.read_csv('/advdata/train.csv')
Testdf = pd.read_csv('/advdata/test.csv')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Traindf.head()

,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,ConversionStatus,ConversionDate,ConversionPayOut,publisherId,subPublisherId,advertiserCampaignId,Fraud
0,0,IN,2.0,M,2017-08-21 00:00:01,Google,chrome,Android,NaN,42.111.171.131,False,NaN,0.0,69,NaN,6555.0,0.0
1,1,TH,27.0,A,2017-08-21 00:00:01,Generic,46.0.2490.76,Android,NaN,1.46.235.65,False,NaN,0.0,646,NaN,7510.0,0.0
2,2,IN,4.0,M,2017-08-21 00:00:01,Samsung,chromium,Android,NaN,49.15.91.224,False,NaN,0.0,301,NaN,12696.0,0.0
3,3,PH,-1.0,A,2017-08-21 00:00:01,Generic,android_webkit,Android,NaN,115.42.126.86,False,NaN,0.0,242,NaN,735.0,0.0
4,4,MX,-1.0,NaN,2017-08-21 00:00:01,Generic,chrome,Android,NaN,187.161.11.97,False,NaN,0.0,1306,NaN,7802.0,0.0


In [4]:
print ('The shape of the Full training Data is: ', Traindf.shape)
print ('The shape of the Full Test Data is: ', Testdf.shape)
ConvTrndf = Traindf[Traindf.ConversionStatus==True]
print ('The shape of the dataframe with only converted members is: ', ConvTrndf.shape)
print ('The Percentage of conversion is: ', ((ConvTrndf.shape[0] / Traindf.shape[0])*100))

The shape of the Full training Data is:  (63367217, 17)
The shape of the Full Test Data is:  (25548873, 14)
The shape of the dataframe with only converted members is:  (34524, 17)
The Percentage of conversion is:  0.054482430560269045


In [5]:
del ConvTrndf

In [7]:
# this indicates in the complete sample that we have taken there are Fraud variable set to 1.0
print ('Fraud Count in Train',Traindf.Fraud.value_counts())
print ('Fraud Count in Test',Testdf.Fraud.value_counts())

Fraud Count in Train 0.0    63366821
1.0         396
Name: Fraud, dtype: int64
Fraud Count in Test 0.0    25444662
1.0      104211
Name: Fraud, dtype: int64


In [8]:
# Convert the Fraud and Conversion Status to integer 
Traindf['Fraud'] = Traindf['Fraud'].astype(np.int)
Testdf['Fraud'] = Testdf['Fraud'].astype(np.int)
Traindf['ConversionStatus'] = Traindf['ConversionStatus'].astype(np.int)

In [9]:
# Checking for the ratio of missing value
Traindf_na = (Traindf.isnull().sum() / len(Traindf)) * 100
Traindf_na = Traindf_na.drop(Traindf_na[Traindf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Traindf_na})
missing_data.head(10)

,Missing Ratio
ConversionDate,99.945518
subPublisherId,98.348556
RefererUrl,74.716878
TrafficType,14.978493
OS,0.487481
Browser,0.430606
Device,0.209716
Country,0.064795
advertiserCampaignId,0.000726
publisherId,0.000314


In [10]:
del Traindf_na, missing_data

In [11]:
# Checking for the ratio of missing value
Testdf_na = (Testdf.isnull().sum() / len(Testdf)) * 100
Testdf_na = Testdf_na.drop(Testdf_na[Testdf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Testdf_na})
missing_data.head(10)

,Missing Ratio
subPublisherId,97.288667
RefererUrl,63.835008
TrafficType,20.355023
OS,0.521236
Browser,0.372200
Device,0.214311
Country,0.035375
advertiserCampaignId,0.001558
publisherId,0.000595


In [12]:
del Testdf_na, missing_data

In [13]:
Traindf.TrafficType.value_counts().head()

A    36659661
M    17216102
Name: TrafficType, dtype: int64

In [16]:
# Replace the Missing value based on above result
#Traindf['Country'] = Traindf.Country.fillna('IN').astype(str)
#Traindf['Device'] = Traindf.Device.fillna('UN').astype(str)
Traindf['publisherId'] = Traindf.publisherId.fillna(1306).astype(str)
#Traindf['advertiserCampaignId'] = Traindf.advertiserCampaignId.fillna(7510).astype(np.int)
#Testdf['Country'] = Testdf.Country.fillna('IN').astype(str)
#Testdf['Device'] = Testdf.Device.fillna('UN').astype(str)
Testdf['publisherId'] = Testdf.publisherId.fillna(1306).astype(str)
#Testdf['advertiserCampaignId'] = Testdf.advertiserCampaignId.fillna(7510).astype(np.int)

In [17]:
# Combining the data only for the necessary fields
STrain = Traindf[['Country','Carrier','Device','publisherId','advertiserCampaignId','Fraud']]
STest = Testdf[['Country','Carrier','Device','publisherId','advertiserCampaignId','Fraud']]
Combdf = pd.concat([STrain,STest])
Combdf.shape

(88916090, 6)

In [34]:
#del STrain, STest
del Traindf, Testdf

In [19]:
Combdf.head()

,Country,Carrier,Device,publisherId,advertiserCampaignId,Fraud
0,IN,2.0,Google,69,6555,0
1,TH,27.0,Generic,646,7510,0
2,IN,4.0,Samsung,301,12696,0
3,PH,-1.0,Generic,242,735,0
4,MX,-1.0,Generic,1306,7802,0


In [24]:
# Count the count of the variables which we should convert into their counts and run it one varaible at a time
#Combdf['CntryCount'] = Combdf.groupby('Country')['Country'].transform('count').astype(np.int)
#Combdf['CarrCount'] = Combdf.groupby('Carrier')['Carrier'].transform('count').astype(np.int)
#Combdf['DevCount'] = Combdf.groupby('Device')['Device'].transform('count').astype(np.int)
#Combdf['PubCount'] = Combdf.groupby('publisherId')['publisherId'].transform('count').astype(np.int)
Combdf['AdvCount'] = Combdf.groupby('advertiserCampaignId')['advertiserCampaignId'].transform('count').astype(np.int)

In [31]:
# Count the count of the variables which we should convert into their counts and run it one varaible at a time
#Combdf['CntryFrd'] = Combdf.groupby('Country')['Fraud'].transform('sum').astype(np.int)
#Combdf['CarrFrd'] = Combdf.groupby('Carrier')['Fraud'].transform('sum').astype(np.int)
#Combdf['DevFrd'] = Combdf.groupby('Device')['Fraud'].transform('sum').astype(np.int)
#Combdf['PubFrd'] = Combdf.groupby('publisherId')['Fraud'].transform('sum').astype(np.int)
Combdf['AdvFrd'] = Combdf.groupby('advertiserCampaignId')['Fraud'].transform('sum').astype(np.int)

In [29]:
#Identify the min, 33rd, 66th and max percentile in data for all Count columns
print ('Min is',Combdf.CntryCount.min(),',33 percentile is', np.percentile(Combdf.CntryCount, 33), ',66 percentile is', np.percentile(Combdf.CntryCount, 66), ',Max Value is', Combdf.CntryCount.max())
print ('Min is',Combdf.CarrCount.min(),',33 percentile is', np.percentile(Combdf.CarrCount, 33), ',66 percentile is', np.percentile(Combdf.CarrCount, 66), ',Max Value is', Combdf.CarrCount.max())
print ('Min is',Combdf.DevCount.min(),',33 percentile is', np.percentile(Combdf.DevCount, 33), ',66 percentile is', np.percentile(Combdf.DevCount, 66), ',Max Value is', Combdf.DevCount.max())
print ('Min is',Combdf.PubCount.min(),',33 percentile is', np.percentile(Combdf.PubCount, 33), ',66 percentile is', np.percentile(Combdf.PubCount, 66), ',Max Value is', Combdf.PubCount.max())
print ('Min is',Combdf.AdvCount.min(),',33 percentile is', np.percentile(Combdf.AdvCount, 33), ',66 percentile is', np.percentile(Combdf.AdvCount, 66), ',Max Value is', Combdf.AdvCount.max())

Min is 1 ,33 percentile is 2721633.0 ,66 percentile is 8536822.0 ,Max Value is 27137013
Min is 1 ,33 percentile is 1776906.0 ,66 percentile is 6282011.0 ,Max Value is 27640537
Min is 1 ,33 percentile is 3492463.0 ,66 percentile is 43168936.0 ,Max Value is 43168936
Min is 1 ,33 percentile is 390083.0 ,66 percentile is 1929233.0 ,Max Value is 6407143
Min is 1 ,33 percentile is 731465.0 ,66 percentile is 2684511.0 ,Max Value is 6662847


In [32]:
#Identify the min, 33rd, 66th and max percentile in data for all Fraud Sum columns
print ('Min is',Combdf.CntryFrd.min(),',Median is', np.percentile(Combdf.CntryFrd, 5), ',80 percentile is', np.percentile(Combdf.CntryFrd, 80), ',Max Value is', Combdf.CntryFrd.max())
print ('Min is',Combdf.CarrFrd.min(),',Median is', np.percentile(Combdf.CarrFrd, 50), ',80 percentile is', np.percentile(Combdf.CarrFrd, 80), ',Max Value is', Combdf.CarrFrd.max())
print ('Min is',Combdf.DevFrd.min(),',Median is', np.percentile(Combdf.DevFrd, 5), ',80 percentile is', np.percentile(Combdf.DevFrd, 80), ',Max Value is', Combdf.DevFrd.max())
print ('Min is',Combdf.PubFrd.min(),',Median is', np.percentile(Combdf.PubFrd, 50), ',80 percentile is', np.percentile(Combdf.PubFrd, 80), ',Max Value is', Combdf.PubFrd.max())
print ('Min is',Combdf.AdvFrd.min(),',Median is', np.percentile(Combdf.AdvFrd, 5), ',80 percentile is', np.percentile(Combdf.AdvFrd, 80), ',Max Value is', Combdf.AdvFrd.max())

Min is 0 ,Median is 23.0 ,80 percentile is 9986.0 ,Max Value is 26775
Min is 0 ,Median is 1233.0 ,80 percentile is 32729.0 ,Max Value is 32729
Min is 0 ,Median is 103.0 ,80 percentile is 46748.0 ,Max Value is 46748
Min is 0 ,Median is 22.0 ,80 percentile is 3398.0 ,Max Value is 20675
Min is 0 ,Median is 0.0 ,80 percentile is 6430.0 ,Max Value is 11386


In [1]:
#Countrydf = Combdf[['Country','CntryCount','CntryFrd']]
#Carrierdf = Combdf[['Carrier','CarrCount','CarrFrd']]
#Devicedf = Combdf[['Device','DevCount','DevFrd']]
#Publisherdf = Combdf[['publisherId','PubCount','PubFrd']]
#Advertiserdf = Combdf[['advertiserCampaignId','AdvCount','AdvFrd']]

In [34]:
print (Countrydf.shape)
print (Carrierdf.shape)
print (Devicedf.shape)
print (Publisherdf.shape)
print (Advertiserdf.shape)

(88916090, 3)
(88916090, 3)
(88916090, 3)
(88916090, 3)
(88916090, 3)


In [35]:
del Combdf

In [43]:
Countrydf = Countrydf.drop_duplicates()
Carrierdf = Carrierdf.drop_duplicates()
Devicedf = Devicedf.drop_duplicates()
Publisherdf = Publisherdf.drop_duplicates()
Advertiserdf = Advertiserdf.drop_duplicates()

In [44]:
print (Countrydf.shape)
print (Carrierdf.shape)
print (Devicedf.shape)
print (Publisherdf.shape)
print (Advertiserdf.shape)

(232, 3)
(383, 3)
(1344, 3)
(12534, 3)
(2132, 3)


In [47]:
#Countrydf.to_csv('Countrydf.csv', index= False)
#Carrierdf.to_csv('Carrierdf.csv', index= False)
#Devicedf.to_csv('Devicedf.csv', index= False)
#Publisherdf.to_csv('Publisherdf.csv', index= False)
Advertiserdf.to_csv('Advertiserdf.csv', index= False)

In [48]:
print ('Shape of training data Before merging the new cols',Traindf.shape)
print ('Shape of Test data Before merging the new cols',Testdf.shape)

Shape of training data Before merging the new cols (63367217, 17)
Shape of Test data Before merging the new cols (25548873, 14)


In [59]:
print ('Shape of training data Before merging the new cols',Traindf.shape)
print ('Shape of Test data Before merging the new cols',Testdf.shape)

Shape of training data Before merging the new cols (63367217, 17)
Shape of Test data Before merging the new cols (25548873, 24)


# Restart without the Count of variables

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 999)

In [2]:
# Importing the part df on local and full data on cloud
#filename = '/Users/viswanathravindran/AVDatahack/ADClick/RawData/train.csv'
#Traindf = pd.read_csv(filename, nrows = 20000)
Traindf = pd.read_csv('/advdata/train.csv')
#Testdf = pd.read_csv('/advdata/test.csv')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Traindf.head()

,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,ConversionStatus,ConversionDate,ConversionPayOut,publisherId,subPublisherId,advertiserCampaignId,Fraud
0,0,IN,2.0,M,2017-08-21 00:00:01,Google,chrome,Android,NaN,42.111.171.131,False,NaN,0.0,69,NaN,6555.0,0.0
1,1,TH,27.0,A,2017-08-21 00:00:01,Generic,46.0.2490.76,Android,NaN,1.46.235.65,False,NaN,0.0,646,NaN,7510.0,0.0
2,2,IN,4.0,M,2017-08-21 00:00:01,Samsung,chromium,Android,NaN,49.15.91.224,False,NaN,0.0,301,NaN,12696.0,0.0
3,3,PH,-1.0,A,2017-08-21 00:00:01,Generic,android_webkit,Android,NaN,115.42.126.86,False,NaN,0.0,242,NaN,735.0,0.0
4,4,MX,-1.0,NaN,2017-08-21 00:00:01,Generic,chrome,Android,NaN,187.161.11.97,False,NaN,0.0,1306,NaN,7802.0,0.0


In [4]:
# Convert the Fraud and Conversion Status to integer 
Traindf['Fraud'] = Traindf['Fraud'].astype(np.int)
Traindf['ConversionStatus'] = Traindf['ConversionStatus'].astype(np.int)

In [5]:
del Traindf['subPublisherId']

In [6]:
Traindf.ConversionStatus.value_counts()

0    63332693
1       34524
Name: ConversionStatus, dtype: int64

In [7]:
# Seperating the Classes 
TraindfY = Traindf[(Traindf['ConversionStatus'] == 1)]
TraindfN = Traindf[(Traindf['ConversionStatus'] == 0)]
print ('Count of Status of Y', TraindfY.shape, 'Count of Status of N', TraindfN.shape)

Count of Status of Y (34524, 16) Count of Status of N (63332693, 16)


In [8]:
del Traindf

In [9]:
34524/ (34524 + 110000)

0.23888073953115055

In [10]:
34524 + 110000

144524

In [11]:
# Select a sample so that the ratio is near 80:20 of both the class
TraindfN = TraindfN.sample(n=110000)
print ('After sampling the shape of TraindfN1 is', TraindfN.shape)

After sampling the shape of TraindfN1 is (110000, 16)


In [12]:
TraindfN.TrafficType.value_counts()

A    63673
M    29787
Name: TrafficType, dtype: int64

In [13]:
# Merging the Traindf after resampling
Traindf = pd.concat([TraindfY, TraindfN])
print ('The Shape of the New training dataframe is:', Traindf.shape)

The Shape of the New training dataframe is: (144524, 16)


In [14]:
# delete the 2 temp DF's
del TraindfY, TraindfN

In [15]:
# Check to see if the sample is fine
#Traindf.tail()
#Traindf.Country.value_counts(normalize =  True, sort=True, ascending=False).head(5)
#Traindf.TrafficType.value_counts(normalize =  True, sort=True, ascending=False).head()
#Traindf.Device.value_counts(normalize =  True, sort=True, ascending=False).head()
Traindf.ConversionStatus.value_counts()

0    110000
1     34524
Name: ConversionStatus, dtype: int64

In [16]:
# Change the Referral URL as either Yes or No
Traindf['RefererUrl'] = np.where(Traindf['RefererUrl']=='NoRefer', 0, 1).astype(int)

In [17]:
# Handing the ClickDate field as a Date
Traindf['ClickDate'] = pd.to_datetime(Traindf['ClickDate'])

In [18]:
# Feature Enginerring to Create New variables for Day of Week and Day part
Traindf['Weekday'] = (Traindf['ClickDate'].dt.dayofweek).astype(int)
Traindf['Dayhour'] = (Traindf.ClickDate.apply(lambda x: x.hour)).astype(int)

In [19]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
Traindf['Weekday'] = Traindf['Weekday'].map( {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed',
                                              4: 'Thr', 5: 'Fri', 6: 'Sat'} ).astype(str)
Traindf['Dayhour'] = Traindf['Dayhour'].map( {0: 'PrAM', 1: 'PrAM', 2: 'PrAM', 3: 'PrAM', 4: 'PrAM', 5: 'PrAM', 
                                              6: 'AM', 7: 'AM', 8: 'AM', 9: 'AM', 10: 'AM', 11: 'AM', 12: 'PM', 
                                              13: 'PM', 14: 'PM', 15: 'PM', 16: 'PM', 17: 'PM', 18: 'PM', 
                                              19: 'PoPM', 20: 'PoPM', 21: 'PoPM', 22: 'PoPM', 23: 'PoPM'} ).astype(str)
# Delete the Click Date varaible
del Traindf['ClickDate']

In [20]:
Traindf.shape

(144524, 17)

In [21]:
Traindf.head()

,ID,Country,Carrier,TrafficType,Device,Browser,OS,RefererUrl,UserIp,ConversionStatus,ConversionDate,ConversionPayOut,publisherId,advertiserCampaignId,Fraud,Weekday,Dayhour
1378,1378,BD,-1.0,A,Generic,46.0.2490.76,Android,1,123.108.246.34,1,2017-08-21 20:40:00,4.550000,520,4904.0,0,Sun,PrAM
1720,1720,BD,44.0,A,Samsung,chrome,Android,1,119.30.32.128,1,2017-08-21 00:00:00,2.652650,274,4129.0,0,Sun,PrAM
3698,3698,RU,703.0,A,Alcatel,chrome,Android,1,83.149.46.223,1,2017-08-21 02:22:00,25.493000,301,11829.0,0,Sun,PrAM
4050,4050,BD,44.0,A,Generic,android_webkit,Android,1,119.30.47.26,1,2017-08-21 01:07:00,1.123456,6240,3312.0,0,Sun,PrAM
8667,8667,IN,1.0,A,Intex,chrome,Android,1,223.182.190.14,1,2017-08-21 00:35:00,65.000000,209,16060.0,0,Sun,PrAM


In [22]:
Traindf.to_csv('AfterSample.csv', index= False)

# Restart after sampling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 999)

In [23]:
Traindf = pd.read_csv('AfterSample.csv')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
Traindf.shape

(144524, 17)

In [25]:
# Replace the Missing value based on analysis
Traindf['Fraud'] = Traindf['Fraud'].astype(np.int)
Traindf['ConversionStatus'] = Traindf['ConversionStatus'].astype(np.int)
Traindf['Country'] = Traindf.Country.fillna('IN').astype(str)
Traindf['Device'] = Traindf.Device.fillna('UN').astype(str)
Traindf['publisherId'] = Traindf.publisherId.fillna(1306).astype(str)
Traindf['advertiserCampaignId'] = Traindf.advertiserCampaignId.fillna(7510).astype(np.int)
Traindf['TrafficType'] = Traindf.TrafficType.fillna('UN').astype(str)
Traindf['Browser'] = Traindf.Browser.fillna('android_webkit').astype(str)
Traindf['OS'] = Traindf.OS.fillna('Android').astype(str)
Traindf['RefererUrl'] = Traindf.RefererUrl.fillna('NoRefer').astype(str)

In [26]:
# Checking for the ratio of missing value
Traindf_na = (Traindf.isnull().sum() / len(Traindf)) * 100
Traindf_na = Traindf_na.drop(Traindf_na[Traindf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Traindf_na})
missing_data.head(10)

,Missing Ratio
ConversionDate,76.111926
ConversionPayOut,0.001384


In [27]:
#Traindf['publisherId_New']=Traindf.publisherId.apply(lambda x: np.where(x.isdigit(),x,'99999')).astype(int)
del Traindf['publisherId']

In [28]:
# Replacing the country with their Area
url="https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regiondf = pd.read_csv(url, usecols=['alpha-2','region'])
regiondf.columns = ['Country', 'Region']

In [29]:
regiondf['Region'] = regiondf.Region.fillna('Unknown')
regiondf = regiondf[pd.notnull(regiondf['Country'])]

In [30]:
Traindf = pd.merge(Traindf,regiondf, how='left', left_on='Country', right_on='Country')

In [31]:
# Code to check the value of the missing field
# miss = Traindf[Traindf['Region'].isnull()]
Traindf.loc[(Traindf['Country'] == 'UK') ,'Region'] = 'Europe'
Traindf.loc[(Traindf['Country'] == '**') ,'Region'] = 'Asia'

In [32]:
Traindf.shape

(144524, 17)

In [33]:
Traindf.head()

,ID,Country,Carrier,TrafficType,Device,Browser,OS,RefererUrl,UserIp,ConversionStatus,ConversionDate,ConversionPayOut,advertiserCampaignId,Fraud,Weekday,Dayhour,Region
0,1378,BD,-1.0,A,Generic,46.0.2490.76,Android,1,123.108.246.34,1,2017-08-21 20:40:00,4.550000,4904,0,Sun,PrAM,Asia
1,1720,BD,44.0,A,Samsung,chrome,Android,1,119.30.32.128,1,2017-08-21 00:00:00,2.652650,4129,0,Sun,PrAM,Asia
2,3698,RU,703.0,A,Alcatel,chrome,Android,1,83.149.46.223,1,2017-08-21 02:22:00,25.493000,11829,0,Sun,PrAM,Europe
3,4050,BD,44.0,A,Generic,android_webkit,Android,1,119.30.47.26,1,2017-08-21 01:07:00,1.123456,3312,0,Sun,PrAM,Asia
4,8667,IN,1.0,A,Intex,chrome,Android,1,223.182.190.14,1,2017-08-21 00:35:00,65.000000,16060,0,Sun,PrAM,Asia


In [34]:
Countrydf = pd.read_csv('Countrydf.csv')
Carrierdf = pd.read_csv('Carrierdf.csv')
Devicedf = pd.read_csv('Devicedf.csv')
Publisherdf = pd.read_csv('Publisherdf.csv')
Advertiserdf = pd.read_csv('Advertiserdf.csv')

In [35]:
Publisherdf['publisherId']=Publisherdf.publisherId.astype(str)

In [36]:
#Publisherdf['publisherId_New']=Publisherdf.publisherId.apply(lambda x: np.where(x.isdigit(),x,'99999')).astype(int)
del Publisherdf['publisherId']

In [37]:
# Merge back the Columns to train and test
Traindf = pd.merge(Traindf,Countrydf, how='left', left_on='Country', right_on='Country')
Traindf.shape

(144524, 19)

In [38]:
Traindf = pd.merge(Traindf,Carrierdf, how='left', left_on='Carrier', right_on='Carrier')
Traindf.shape

(144524, 21)

In [39]:
Traindf = pd.merge(Traindf,Devicedf, how='left', left_on='Device', right_on='Device')
Traindf.shape

(144524, 23)

In [40]:
#Traindf = pd.merge(Traindf,Publisherdf, how='left', left_on='publisherId_New', right_on='publisherId_New')
#Traindf.shape

In [41]:
Traindf = pd.merge(Traindf,Advertiserdf, how='left', left_on='advertiserCampaignId', right_on='advertiserCampaignId')
Traindf.shape

(144524, 25)

In [42]:
Traindf.head()

,ID,Country,Carrier,TrafficType,Device,Browser,OS,RefererUrl,UserIp,ConversionStatus,ConversionDate,ConversionPayOut,advertiserCampaignId,Fraud,Weekday,Dayhour,Region,CntryCount,CntryFrd,CarrCount,CarrFrd,DevCount,DevFrd,AdvCount,AdvFrd
0,1378,BD,-1.0,A,Generic,46.0.2490.76,Android,1,123.108.246.34,1,2017-08-21 20:40:00,4.550000,4904,0,Sun,PrAM,Asia,8191723,26775,27640537.0,32729,43168936,46748,4453198.0,3126
1,1720,BD,44.0,A,Samsung,chrome,Android,1,119.30.32.128,1,2017-08-21 00:00:00,2.652650,4129,0,Sun,PrAM,Asia,8191723,26775,2108346.0,6847,13575659,14293,1081260.0,5071
2,3698,RU,703.0,A,Alcatel,chrome,Android,1,83.149.46.223,1,2017-08-21 02:22:00,25.493000,11829,0,Sun,PrAM,Europe,3349617,16756,924488.0,3428,3492463,11144,1704306.0,5538
3,4050,BD,44.0,A,Generic,android_webkit,Android,1,119.30.47.26,1,2017-08-21 01:07:00,1.123456,3312,0,Sun,PrAM,Asia,8191723,26775,2108346.0,6847,43168936,46748,5745057.0,6430
4,8667,IN,1.0,A,Intex,chrome,Android,1,223.182.190.14,1,2017-08-21 00:35:00,65.000000,16060,0,Sun,PrAM,Asia,27137013,9986,6282011.0,329,1390903,560,502408.0,1


In [22]:
#miss = Traindf[Traindf['PubCount'].isnull()]
#miss.shape

In [43]:
# Creating a Category as Android Yes / No
Traindf['AndroidOS'] = np.where(Traindf['OS']=='Android', 'yes', 'no')
del Traindf['OS']

In [44]:
# Convert the User Ip Address as Classes that they belong
Traindf['FirstIP'] = Traindf['UserIp'].str.split('.').str[0]
Traindf['FirstIP'] = Traindf['FirstIP'].astype(int)

def func1(row):
    if row['FirstIP'] <= 128:
        val = 'ClassA'
    elif row['FirstIP'] <= 191:
        val = 'ClassB'
    elif row['FirstIP'] <= 223:
        val = 'ClassC'
    elif row['FirstIP'] <= 239:
        val = 'ClassD'
    elif row['FirstIP'] <= 255:
        val = 'ClassE'
    else:
        val = 'NoClass'
    return val

Traindf['IPClass'] = Traindf.apply(func1, axis = 1).astype(str)
del Traindf['FirstIP'], Traindf['UserIp']

In [45]:
# Cleaning the Browser column and creation only 3 categories based on previous cell 
Traindf['Browser'] = Traindf['Browser'].astype(str)

def func2(row):
    if row['Browser'] == 'android_webkit':
        val = 'AndBrowser'
    elif row['Browser'] == 'chrome':
        val = 'Chrome'
    elif row['Browser'] == 'chromium':
        val = 'Chrome'
    elif row['Browser'] == 'firefox_desktop':
        val = 'firefox'
    elif row['Browser'] == 'firefox_mobile':
        val = 'firefox'
    elif row['Browser'] == 'iphone':
        val = 'Safari'
    elif row['Browser'] == 'safari':
        val = 'Safari'
    elif row['Browser'] == 'blackberry':
        val = 'blackberry'
    elif row['Browser'] == 'opera':
        val = 'opera'
    elif row['Browser'] == 'iemobile':
        val = 'iemobile'
    else:
        val = 'Others'
    return val

Traindf['Browser'] = Traindf.apply(func2, axis = 1).astype(str)

In [46]:
# Remove the Y's
All_y = Traindf[['ID', 'ConversionStatus','ConversionDate','ConversionPayOut']]

In [47]:
# Removing the variables that are part of Y
del Traindf['ConversionStatus'],Traindf['ConversionDate'],Traindf['ConversionPayOut']

In [48]:
# Remove unnecessary varaibles
del Traindf['Country'],Traindf['Carrier'],Traindf['Device'],Traindf['advertiserCampaignId']

In [49]:
def func1(row):
    if row['CntryCount'] <= 2721633.0:
        val = 'L'
    elif row['CntryCount'] <= 8536822.0:
        val = 'M'
    else:
        val = 'H'
    return val

def func2(row):
    if row['CarrCount'] <= 1776906.0:
        val = 'L'
    elif row['CarrCount'] <= 6282011.0:
        val = 'M'
    else:
        val = 'H'
    return val

def func3(row):
    if row['DevCount'] <= 3492463.0:
        val = 'L'
    elif row['DevCount'] <= 43168936.0:
        val = 'M'
    else:
        val = 'H'
    return val

def func4(row):
    if row['PubCount'] <= 433397.0:
        val = 'L'
    elif row['PubCount'] <= 2719241.0:
        val = 'M'
    else:
        val = 'H'
    return val

def func5(row):
    if row['AdvCount'] <= 731465.0:
        val = 'L'
    elif row['AdvCount'] <= 2684511.0:
        val = 'M'
    else:
        val = 'H'
    return val

Traindf['CntryCntClass'] = Traindf.apply(func1, axis = 1).astype(str)
Traindf['TraficCntClass'] = Traindf.apply(func2, axis = 1).astype(str)
Traindf['DevCntClass'] = Traindf.apply(func3, axis = 1).astype(str)
#Traindf['PubCntClass'] = Traindf.apply(func4, axis = 1).astype(str)
Traindf['AdtsrCntClass'] = Traindf.apply(func5, axis = 1).astype(str)

del Traindf['CntryCount'],Traindf['CarrCount'],Traindf['DevCount'],Traindf['AdvCount']
#Traindf['PubCount'],

In [50]:
def func1(row):
    if row['CntryFrd'] <= 23.0:
        val = 'LF'
    elif row['CntryFrd'] <= 9986.0:
        val = 'MF'
    else:
        val = 'HF'
    return val

def func2(row):
    if row['CarrFrd'] <= 1233.0:
        val = 'LF'
    elif row['CarrFrd'] <= 32729.0:
        val = 'MF'
    else:
        val = 'HF'
    return val

def func3(row):
    if row['DevFrd'] <= 46748.0:
        val = 'LF'
    else:
        val = 'HF'
    return val

def func4(row):
    if row['PubFrd'] <= 11.0:
        val = 'LF'
    else:
        val = 'HF'
    return val

def func5(row):
    if row['AdvFrd'] <= 6430.0:
        val = 'LF'
    else:
        val = 'HF'
    return val

Traindf['CntryFrdClass'] = Traindf.apply(func1, axis = 1).astype(str)
Traindf['TraficFrdClass'] = Traindf.apply(func2, axis = 1).astype(str)
Traindf['DevFrdClass'] = Traindf.apply(func3, axis = 1).astype(str)
#Traindf['PubFrdClass'] = Traindf.apply(func4, axis = 1).astype(str)
Traindf['AdtsrFrdClass'] = Traindf.apply(func5, axis = 1).astype(str)

del Traindf['CntryFrd'],Traindf['CarrFrd'],Traindf['DevFrd'],Traindf['AdvFrd']
#Traindf['PubFrd'],

In [51]:
Traindf.shape

(144524, 18)

In [52]:
Traindf.head()

,ID,TrafficType,Browser,RefererUrl,Fraud,Weekday,Dayhour,Region,AndroidOS,IPClass,CntryCntClass,TraficCntClass,DevCntClass,AdtsrCntClass,CntryFrdClass,TraficFrdClass,DevFrdClass,AdtsrFrdClass
0,1378,A,Others,1,0,Sun,PrAM,Asia,yes,ClassA,M,H,M,H,HF,MF,LF,LF
1,1720,A,Chrome,1,0,Sun,PrAM,Asia,yes,ClassA,M,M,M,M,HF,MF,LF,LF
2,3698,A,Chrome,1,0,Sun,PrAM,Europe,yes,ClassA,M,L,L,M,HF,MF,LF,LF
3,4050,A,AndBrowser,1,0,Sun,PrAM,Asia,yes,ClassA,M,M,M,H,HF,MF,LF,LF
4,8667,A,Chrome,1,0,Sun,PrAM,Asia,yes,ClassC,H,M,L,L,MF,LF,LF,LF


In [53]:
Traindf.columns = ['ID','TrafficType','Browser','RefererUrl','Fraud','Weekday','Dayhour','Region', 'AndroidOS',
                   'IPClass','CntryCntClass','TraficCntClass','DevCntClass','AdtsrCntClass',
                   'CntryFrdClass','TraficFrdClass','DevFrdClass','AdtsrFrdClass']
#'PubCntClass','PubFrdClass',

In [54]:
# Verify the variables for null
Traindf_na = (Traindf.isnull().sum() / len(Traindf)) * 100
Traindf_na = Traindf_na.drop(Traindf_na[Traindf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Traindf_na})
missing_data.head(10)

,Missing Ratio
Region,0.000692


In [55]:
miss = Traindf[Traindf['Region'].isnull()]
miss

,ID,TrafficType,Browser,RefererUrl,Fraud,Weekday,Dayhour,Region,AndroidOS,IPClass,CntryCntClass,TraficCntClass,DevCntClass,AdtsrCntClass,CntryFrdClass,TraficFrdClass,DevFrdClass,AdtsrFrdClass
72249,2181728,UN,AndBrowser,1,0,Sun,PM,NaN,yes,ClassA,L,H,L,L,LF,MF,LF,LF


In [57]:
Traindf['Region'] = Traindf.Region.fillna('Europe').astype(str)

In [58]:
Traindf.to_csv('ClassifierTrain.csv', index= False)

In [59]:
All_y.to_csv('All_y.csv', index=False)

In [60]:
Traindf.shape

(144524, 18)

In [62]:
#del Traindf
del All_y

# Convert the Test data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 999)

In [2]:
#filename = '/Users/viswanathravindran/AVDatahack/ADClick/RawData/test.csv'
#Testdf = pd.read_csv(filename, nrows=50000)
Testdf = pd.read_csv('/advdata/test.csv')

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Testdf.shape

(25548873, 14)

In [4]:
Testdf.head()

,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,publisherId,subPublisherId,advertiserCampaignId,Fraud
0,63367289,IN,5.0,NaN,2017-08-31 00:00:01,Generic,uc,Android,NaN,101.58.211.15,1479,NaN,16216.0,0.0
1,63367290,IN,2.0,M,2017-08-31 00:00:01,Datawind,chrome,Android,NaN,42.107.175.145,3734,NaN,1836.0,0.0
2,63367291,IN,1.0,A,2017-08-31 00:00:01,Panasonic,chrome,Android,synctraff.com,106.223.85.251,282,NaN,13431.0,0.0
3,63367292,BR,-1.0,M,2017-08-31 00:00:01,Generic,android_webkit,Android,NaN,177.35.22.162,1203,NaN,4787.0,0.0
4,63367293,IN,3.0,A,2017-08-31 00:00:01,Micromax,chrome,Android,NaN,101.210.255.232,597,NaN,9769.0,0.0


In [13]:
# Replace the Missing value based on analysis
#Testdf['Fraud'] = Testdf['Fraud'].astype(np.int)
#Testdf['Country'] = Testdf.Country.fillna('IN').astype(str)
#Testdf['Device'] = Testdf.Device.fillna('UN').astype(str)
#Testdf['publisherId'] = Testdf.publisherId.fillna(1306).astype(np.int)
#Testdf['advertiserCampaignId'] = Testdf.advertiserCampaignId.fillna(7510).astype(np.int)
#Testdf['TrafficType'] = Testdf.TrafficType.fillna('UN').astype(str)
#Testdf['Browser'] = Testdf.Browser.fillna('android_webkit').astype(str)
#Testdf['OS'] = Testdf.OS.fillna('Android').astype(str)
Testdf['RefererUrl'] = Testdf.RefererUrl.fillna('NoRefer').astype(str)

In [14]:
# Checking for the ratio of missing value
Testdf_na = (Testdf.isnull().sum() / len(Testdf)) * 100
Testdf_na = Testdf_na.drop(Testdf_na[Testdf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Testdf_na})
missing_data.head(10)

,Missing Ratio
subPublisherId,97.288667
publisherId,0.000595


In [15]:
#Testdf['publisherId_New']=Testdf.publisherId.apply(lambda x: np.where(x.isdigit(),x,'99999')).astype(int)
del Testdf['publisherId'], Testdf['subPublisherId']

In [16]:
# Replacing the country with their Area
url="https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regiondf = pd.read_csv(url, usecols=['alpha-2','region'])
regiondf.columns = ['Country', 'Region']

In [17]:
regiondf['Region'] = regiondf.Region.fillna('Unknown')
regiondf = regiondf[pd.notnull(regiondf['Country'])]

In [18]:
Testdf = pd.merge(Testdf,regiondf, how='left', left_on='Country', right_on='Country')
Testdf.shape

(25548873, 13)

In [19]:
# Code to check the value of the missing field
# miss = Testdf[Testdf['Region'].isnull()]
# miss.shape
# miss
Testdf.loc[(Testdf['Country'] == 'UK') ,'Region'] = 'Europe'
Testdf.loc[(Testdf['Country'] == '**') ,'Region'] = 'Asia'
Testdf.loc[(Testdf['Country'] == 'EU') ,'Region'] = 'Europe'
# del miss

In [20]:
Countrydf = pd.read_csv('Countrydf.csv')
Carrierdf = pd.read_csv('Carrierdf.csv')
Devicedf = pd.read_csv('Devicedf.csv')
Publisherdf = pd.read_csv('Publisherdf.csv')
Advertiserdf = pd.read_csv('Advertiserdf.csv')

In [21]:
Testdf.head()

,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,advertiserCampaignId,Fraud,Region
0,63367289,IN,5.0,UN,2017-08-31 00:00:01,Generic,uc,Android,NoRefer,101.58.211.15,16216,0,Asia
1,63367290,IN,2.0,M,2017-08-31 00:00:01,Datawind,chrome,Android,NoRefer,42.107.175.145,1836,0,Asia
2,63367291,IN,1.0,A,2017-08-31 00:00:01,Panasonic,chrome,Android,synctraff.com,106.223.85.251,13431,0,Asia
3,63367292,BR,-1.0,M,2017-08-31 00:00:01,Generic,android_webkit,Android,NoRefer,177.35.22.162,4787,0,Americas
4,63367293,IN,3.0,A,2017-08-31 00:00:01,Micromax,chrome,Android,NoRefer,101.210.255.232,9769,0,Asia


In [15]:
#Publisherdf['publisherId']=Publisherdf.publisherId.astype(str)

In [16]:
#Publisherdf['publisherId_New']=Publisherdf.publisherId.apply(lambda x: np.where(x.isdigit(),x,'99999')).astype(int)
#del Publisherdf['publisherId']

In [22]:
# Merge back the Columns to train and test
Testdf = pd.merge(Testdf,Countrydf, how='left', left_on='Country', right_on='Country')
Testdf.shape

(25548873, 15)

In [23]:
Testdf = pd.merge(Testdf,Carrierdf, how='left', left_on='Carrier', right_on='Carrier')
Testdf.shape

(25548873, 17)

In [24]:
Testdf = pd.merge(Testdf,Devicedf, how='left', left_on='Device', right_on='Device')
Testdf.shape

(25548873, 19)

In [25]:
#Testdf = pd.merge(Testdf,Publisherdf, how='left', left_on='publisherId_New', right_on='publisherId_New')
#Testdf.shape

In [26]:
Testdf = pd.merge(Testdf,Advertiserdf, how='left', left_on='advertiserCampaignId', right_on='advertiserCampaignId')
Testdf.shape

(25548873, 21)

In [22]:
#miss = Testdf[Traindf['PubCount'].isnull()]
#miss.shape

In [27]:
# Creating a Category as Android Yes / No
Testdf['AndroidOS'] = np.where(Testdf['OS']=='Android', 'yes', 'no')
del Testdf['OS']

In [28]:
# Convert the User Ip Address as Classes that they belong
t0 = time.time()
Testdf['FirstIP'] = Testdf['UserIp'].str.split('.').str[0]
Testdf['FirstIP'] = Testdf['FirstIP'].astype(int)

def func1(row):
    if row['FirstIP'] <= 128:
        val = 'ClassA'
    elif row['FirstIP'] <= 191:
        val = 'ClassB'
    elif row['FirstIP'] <= 223:
        val = 'ClassC'
    elif row['FirstIP'] <= 239:
        val = 'ClassD'
    elif row['FirstIP'] <= 255:
        val = 'ClassE'
    else:
        val = 'NoClass'
    return val

Testdf['IPClass'] = Testdf.apply(func1, axis = 1).astype(str)

t1 = time.time()
print ('Time taken is :',t1 - t0)
del Testdf['FirstIP'], Testdf['UserIp']

In [29]:
# Cleaning the Browser column and creation only 3 categories based on previous cell 
t0 = time.time()
Testdf['Browser'] = Testdf['Browser'].astype(str)

def func2(row):
    if row['Browser'] == 'android_webkit':
        val = 'AndBrowser'
    elif row['Browser'] == 'chrome':
        val = 'Chrome'
    elif row['Browser'] == 'chromium':
        val = 'Chrome'
    elif row['Browser'] == 'firefox_desktop':
        val = 'firefox'
    elif row['Browser'] == 'firefox_mobile':
        val = 'firefox'
    elif row['Browser'] == 'iphone':
        val = 'Safari'
    elif row['Browser'] == 'safari':
        val = 'Safari'
    elif row['Browser'] == 'blackberry':
        val = 'blackberry'
    elif row['Browser'] == 'opera':
        val = 'opera'
    elif row['Browser'] == 'iemobile':
        val = 'iemobile'
    else:
        val = 'Others'
    return val

Testdf['Browser'] = Testdf.apply(func2, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 778.2085852622986


In [30]:
# Remove unnecessary varaibles
del Testdf['Country'],Testdf['Carrier'],Testdf['Device'], Testdf['advertiserCampaignId']

In [31]:
t0 = time.time()
def func1(row):
    if row['CntryCount'] <= 2721633.0:
        val = 'L'
    elif row['CntryCount'] <= 8536822.0:
        val = 'M'
    else:
        val = 'H'
    return val

Testdf['CntryCntClass'] = Testdf.apply(func1, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 580.3456354141235


In [32]:
t0 = time.time()
def func2(row):
    if row['CarrCount'] <= 1776906.0:
        val = 'L'
    elif row['CarrCount'] <= 6282011.0:
        val = 'M'
    else:
        val = 'H'
    return val

Testdf['TraficCntClass'] = Testdf.apply(func2, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 581.8792457580566


In [33]:
t0 = time.time()
def func3(row):
    if row['DevCount'] <= 3492463.0:
        val = 'L'
    elif row['DevCount'] <= 43168936.0:
        val = 'M'
    else:
        val = 'H'
    return val

Testdf['DevCntClass'] = Testdf.apply(func3, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 590.8620352745056


In [34]:
t0 = time.time()
def func4(row):
    if row['PubCount'] <= 433397.0:
        val = 'L'
    elif row['PubCount'] <= 2719241.0:
        val = 'M'
    else:
        val = 'H'
    return val

#Traindf['PubCntClass'] = Traindf.apply(func4, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 0.0001125335693359375


In [35]:
t0 = time.time()
def func5(row):
    if row['AdvCount'] <= 731465.0:
        val = 'L'
    elif row['AdvCount'] <= 2684511.0:
        val = 'M'
    else:
        val = 'H'
    return val

Testdf['AdtsrCntClass'] = Testdf.apply(func5, axis = 1).astype(str)

del Testdf['CntryCount'],Testdf['CarrCount'],Testdf['DevCount'],Testdf['AdvCount']
#Tdel raindf['PubCount']
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 591.617681980133


In [36]:
t0 = time.time()
def func1(row):
    if row['CntryFrd'] <= 23.0:
        val = 'LF'
    elif row['CntryFrd'] <= 9986.0:
        val = 'MF'
    else:
        val = 'HF'
    return val

Testdf['CntryFrdClass'] = Testdf.apply(func1, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 666.7676918506622


In [37]:
t0 = time.time()
def func2(row):
    if row['CarrFrd'] <= 1233.0:
        val = 'LF'
    elif row['CarrFrd'] <= 32729.0:
        val = 'MF'
    else:
        val = 'HF'
    return val

Testdf['TraficFrdClass'] = Testdf.apply(func2, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 549.5853509902954


In [38]:
t0 = time.time()
def func3(row):
    if row['DevFrd'] <= 46748.0:
        val = 'LF'
    else:
        val = 'HF'
    return val

Testdf['DevFrdClass'] = Testdf.apply(func3, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 434.96286940574646


In [39]:
t0 = time.time()
def func4(row):
    if row['PubFrd'] <= 11.0:
        val = 'LF'
    else:
        val = 'HF'
    return val
#Traindf['PubFrdClass'] = Traindf.apply(func4, axis = 1).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 9.72747802734375e-05


In [40]:
t0 = time.time()
def func5(row):
    if row['AdvFrd'] <= 6430.0:
        val = 'LF'
    else:
        val = 'HF'
    return val

Testdf['AdtsrFrdClass'] = Testdf.apply(func5, axis = 1).astype(str)

del Testdf['CntryFrd'],Testdf['CarrFrd'],Testdf['DevFrd'],Testdf['AdvFrd']
#Traindf['PubFrd']
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 436.0487587451935


In [41]:
Testdf.shape

(25548873, 17)

In [42]:
Testdf.head()

,ID,TrafficType,ClickDate,Browser,RefererUrl,Fraud,Region,AndroidOS,IPClass,CntryCntClass,TraficCntClass,DevCntClass,AdtsrCntClass,CntryFrdClass,TraficFrdClass,DevFrdClass,AdtsrFrdClass
0,63367289,UN,2017-08-31 00:00:01,Others,NoRefer,0,Asia,yes,ClassA,H,L,M,L,MF,LF,LF,LF
1,63367290,M,2017-08-31 00:00:01,Chrome,NoRefer,0,Asia,yes,ClassA,H,M,L,L,MF,MF,LF,LF
2,63367291,A,2017-08-31 00:00:01,Chrome,synctraff.com,0,Asia,yes,ClassA,H,M,L,L,MF,LF,LF,LF
3,63367292,M,2017-08-31 00:00:01,AndBrowser,NoRefer,0,Americas,yes,ClassB,M,H,M,L,MF,MF,LF,LF
4,63367293,A,2017-08-31 00:00:01,Chrome,NoRefer,0,Asia,yes,ClassA,H,L,L,L,MF,MF,LF,LF


In [43]:
# Change the Referral URL as either Yes or No
Testdf['RefererUrl'] = np.where(Testdf['RefererUrl']=='NoRefer', 0, 1).astype(int)

In [44]:
# Handing the ClickDate field as a Date
Testdf['ClickDate'] = pd.to_datetime(Testdf['ClickDate'])

In [45]:
# Feature Enginerring to Create New variables for Day of Week and Day part
t0 = time.time()
Testdf['Weekday'] = (Testdf['ClickDate'].dt.dayofweek).astype(int)
Testdf['Dayhour'] = (Testdf.ClickDate.apply(lambda x: x.hour)).astype(int)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 90.39211249351501


In [46]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
t0 = time.time()
Testdf['Weekday'] = Testdf['Weekday'].map( {0: 'Sun', 1: 'Mon', 2: 'Tue', 3: 'Wed',
                                                  4: 'Thr', 5: 'Fri', 6: 'Sat'} ).astype(str)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 5.44592022895813


In [47]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
t0 = time.time()
Testdf['Dayhour'] = Testdf['Dayhour'].map( {0: 'PrAM', 1: 'PrAM', 2: 'PrAM', 3: 'PrAM', 
                                                  4: 'PrAM', 5: 'PrAM', 6: 'AM', 7: 'AM', 8: 'AM', 9: 'AM', 
                                                  10: 'AM', 11: 'AM', 12: 'PM', 13: 'PM', 14: 'PM', 15: 'PM', 
                                                  16: 'PM', 17: 'PM', 18: 'PM', 19: 'PoPM', 20: 'PoPM', 
                                                  21: 'PoPM', 22: 'PoPM', 23: 'PoPM'} ).astype(str)
# Delete the Click Date varaible
del Testdf['ClickDate']
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 5.468459844589233


In [48]:
Testdf.shape

(25548873, 18)

In [49]:
Testdf.head()

,ID,TrafficType,Browser,RefererUrl,Fraud,Region,AndroidOS,IPClass,CntryCntClass,TraficCntClass,DevCntClass,AdtsrCntClass,CntryFrdClass,TraficFrdClass,DevFrdClass,AdtsrFrdClass,Weekday,Dayhour
0,63367289,UN,Others,0,0,Asia,yes,ClassA,H,L,M,L,MF,LF,LF,LF,Wed,PrAM
1,63367290,M,Chrome,0,0,Asia,yes,ClassA,H,M,L,L,MF,MF,LF,LF,Wed,PrAM
2,63367291,A,Chrome,1,0,Asia,yes,ClassA,H,M,L,L,MF,LF,LF,LF,Wed,PrAM
3,63367292,M,AndBrowser,0,0,Americas,yes,ClassB,M,H,M,L,MF,MF,LF,LF,Wed,PrAM
4,63367293,A,Chrome,0,0,Asia,yes,ClassA,H,L,L,L,MF,MF,LF,LF,Wed,PrAM


In [50]:
Testdf = Testdf[['ID','TrafficType','Browser','RefererUrl','Fraud','Weekday','Dayhour','Region', 'AndroidOS',
                   'IPClass','CntryCntClass','TraficCntClass','DevCntClass','AdtsrCntClass',
                   'CntryFrdClass','TraficFrdClass','DevFrdClass','AdtsrFrdClass']]

In [51]:
import sys
Testdf.memory_usage(index=True).sum()

3883428696

In [52]:
# Checking for the missing value details after the replacement
# Checking for the ratio of missing value
Testdf_na = (Testdf.isnull().sum() / len(Testdf)) * 100
Testdf_na = Testdf_na.drop(Testdf_na[Testdf_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :Testdf_na})
missing_data.head(10)

,Missing Ratio


In [53]:
t0 = time.time()
Testdf.to_csv('Finaltest.csv', index=False)
t1 = time.time()
print ('Time taken is :',t1 - t0)

Time taken is : 127.0963306427002


# Modelling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
pd.set_option('display.max_columns', 99)

In [3]:
Traindf = pd.read_csv('/Users/viswanathravindran/AVDatahack/ADClick/ProData/ClassifierTrain.csv')
All_y = pd.read_csv('/Users/viswanathravindran/AVDatahack/ADClick/ProData/All_y.csv')
#Traindf = pd.read_csv('/advdata/FinalClassifTrain.csv')
#All_y = pd.read_csv('/advdata/All_y.csv')

/Users/viswanathravindran/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
Traindf.head()
#Testdf.head()

,ID,TrafficType,Browser,RefererUrl,Fraud,Weekday,Dayhour,Region,AndroidOS,IPClass,CntryCntClass,TraficCntClass,DevCntClass,AdtsrCntClass,CntryFrdClass,TraficFrdClass,DevFrdClass,AdtsrFrdClass
0,1378,A,Others,Android,0,Sun,PrAM,Asia,yes,ClassA,M,H,M,H,HF,MF,LF,LF
1,1720,A,Chrome,Android,0,Sun,PrAM,Asia,yes,ClassA,M,M,M,M,HF,MF,LF,LF
2,3698,A,Chrome,Android,0,Sun,PrAM,Europe,yes,ClassA,M,L,L,M,HF,MF,LF,LF
3,4050,A,AndBrowser,Android,0,Sun,PrAM,Asia,yes,ClassA,M,M,M,H,HF,MF,LF,LF
4,8667,A,Chrome,Android,0,Sun,PrAM,Asia,yes,ClassC,H,M,L,L,MF,LF,LF,LF


In [5]:
print (Traindf.shape)

(144524, 18)


In [6]:
# Convert TrafficType, Browser, OS, Weekday, Dayhour into Numerical
Traindf = pd.get_dummies(Traindf, columns=['TrafficType','Browser','Weekday','Dayhour','Region','AndroidOS',
                                           'IPClass','CntryCntClass','TraficCntClass','DevCntClass','AdtsrCntClass',
                                           'CntryFrdClass','TraficFrdClass','DevFrdClass','AdtsrFrdClass'], drop_first=True)

In [7]:
# Creating y
yTrain = All_y['ConversionStatus'].as_matrix().astype(np.int)

In [9]:
Traindf.RefererUrl.value_counts()

Android             142171
iOS                   1689
Desktop                504
Windows Phone OS       102
Tizen                   30
RIM OS                  21
Mac OS X                 6
Symbian OS               1
Name: RefererUrl, dtype: int64

ValueError: invalid literal for int() with base 10: 'Android'

In [70]:
# Apply Standard Scaling to ensure the variables are in scale
XTest = scaler.fit_transform(XTest)
print (XTest.shape)
XTest

(143937, 45)


/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


array([[ 0.        ,  0.        ,  1.62553502, ...,  0.12590232,
        -0.50744266, -0.39486228],
       [ 0.        ,  0.        ,  1.62553502, ...,  0.12590232,
        -0.50744266, -0.39486228],
       [ 0.        ,  0.        ,  1.62553502, ...,  0.12590232,
        -0.50744266, -0.39486228],
       ..., 
       [ 0.        ,  0.        , -1.41400754, ...,  0.12590232,
        -0.50744266,  2.53252853],
       [ 0.        ,  0.        , -1.41400754, ...,  0.12590232,
         1.97066599, -0.39486228],
       [ 0.        ,  0.        , -1.41400754, ...,  0.12590232,
        -0.50744266, -0.39486228]])

In [71]:
print ('The shape of XTrain is:',XTrain.shape, 'The shape of y is', yTrain.shape)

The shape of X is: (143937, 45) The shape of y is (143937,)


In [72]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
from sklearn import linear_model
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(XTrain, yTrain)

In [87]:
Testdf = pd.read_csv('Finaltest.csv')
#Testdf = pd.read_csv('/advdata/test.csv')

In [89]:
Testdf.shape

In [ ]:
# Convert TrafficType, Browser, OS, Weekday, Dayhour into Numerical
Testdf = pd.get_dummies(Testdf, columns=['TrafficType','Browser','Weekday','Dayhour','Region','AndroidOS',
                                           'IPClass','CntryCntClass','TraficCntClass','DevCntClass','AdtsrCntClass',
                                           'CntryFrdClass','TraficFrdClass','DevFrdClass','AdtsrFrdClass'], drop_first=True)

In [ ]:
print ('The shape of XTest is:',XTest.shape)

In [ ]:
# creating X
del Testdf['ID']
XTest = Testdf.as_matrix().astype(np.int)
XTest

In [ ]:
pred = rfc.precict(XTest)

In [ ]:
NXtest = pd.DataFrame({"ID":Xtest.ID, "y":pred})

In [ ]:
Xtest = pd.merge(Xtest, NXtest, on='ID')

In [ ]:
XTrain = 